Provide the configuration to read data from the S3 bucket (example-eu-central-1)
and save the data to the Databricks schema (students_info). The process includes following steps:
1. Setting up the AWS Credentials
2. Mount S3 bucket to databricks
3. Read data from mounted S3 bucket
4. Save data to schema

**1.Setting up the AWS Credentials**

In [0]:
# Accessing the secrets in AWS : providing AWS access key and secret key to Databricks
# direct configuration
spark.conf.set("fs.s3a.access.key", "YOUR_AWS_ACCESS_KEY")
spark.conf.set("fs.s3a.secret.key", "YOUR_AWS_SECRET_KEY")

# accessing the keys from data bricks secrets scope : providing AWS access key and secret key to Databricks
spark.conf.set("fs.s3a.access.key", dbutils.secrets.get(scope="my-scope", key="aws-access-key"))
spark.conf.set("fs.s3a.secret.key", dbutils.secrets.get(scope="my-scope", key="aws-secret-key"))



 **2. Mount S3 bucket to Databricks**

In [0]:
dbutils.fs.mount(
  source = "s3a://example-eu-central-1",
  mount_point = "/mnt/your-mount-point",
  extra_configs = {"fs.s3a.access.key": dbutils.secrets.get(scope="my-scope", key="aws-access-key"),
                   "fs.s3a.secret.key": dbutils.secrets.get(scope="my-scope", key="aws-secret-key")}
)


**3.Read data from mounted S3 bucket**

In [0]:
#read data from s3 bucket
df = spark.read.format("parquet").option("header", "true").load("/mnt/your-mount-point/path/to/STUDENT.parquet")


**4.Save data to schema**

In [0]:
# Create database schema
spark.sql("CREATE DATABASE IF NOT EXISTS student_info")

# Save DataFrame to a table in the database schema
df.write.mode("overwrite").saveAsTable("student_info.student")